## Questo notebook scarica il calendario earnings da investing.com
## Le date sono settabili come riportato sotto




In [ ]:
%pylab inline
#mounts current gdrive in colab after providing success code
#INSERISCI IL CODICE DI SUCCESSO O NON FUNZIONERA'
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/INVEST-IO/'

Populating the interactive namespace from numpy and matplotlib
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/INVEST-IO


In [ ]:
#apply filter on dates, scarica sempre dallo startdate (da indicare sotto) a enddate
#se enddate = today alla viene scaricato fino ad oggi dinamico, altrimenti imposta enddate alla data desiderata
#!!! FORMATO DATE US !!!
startdate = '05/01/2020'
enddate = '05/31/2020'

#importa la data di oggi nel formato corretto
from datetime import date
import time
if enddate == 'today':
  today = date.today()
  today_us_format = today.strftime("%m/%d/%Y")
else:
  today_us_format = enddate

In [ ]:
%%capture
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as progress_bar
import pandas
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
#function per cliccare pulsanti su chrome .click() di selenium non funziona
def click_on_button_id(x):
  element = wd.find_element_by_id(x)
  wd.execute_script("arguments[0].click();", element)

In [ ]:
#CALENDARIO EARNINGS

url = 'https://www.investing.com/earnings-calendar/'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 4

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

output_list = []
#main scraper loop
tr_tuple = tuple(wd.find_elements_by_xpath('//*[@id="earningsCalendarData"]/tbody//tr'))

In [ ]:
len(tr_tuple)

6682

In [ ]:
for tr in progress_bar(tr_tuple):
  td_tuple = tuple(tr.find_elements_by_css_selector('td'))
  if len(td_tuple) > 1:
    output_list.append({'date' : calendar, 'Comp' : td_tuple[0].text, 'EPS' : td_tuple[1].text,
                        'EPS frc' : td_tuple[2].text, 'Revenues' : td_tuple[3].text, 'Revenues frc' : td_tuple[4].text,
                        'Mkt Cap' : td_tuple[5].text})
  else:
    calendar = td_tuple[0].text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [ ]:
from pandas import DataFrame
df = DataFrame(output_list)
df.to_csv('sr_calendar_earnings_from_' + startdate.replace('/','') +  '_to_' + today_us_format.replace('/', '') + '.csv')

In [ ]:
df.head()